In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import re
import torch
import json_repair
import pandas as pd
from glob import glob 
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer


# 제작한 데이터세 불러오기 
file_list = glob("./data/*.csv")
print(file_list)

df = pd.concat([pd.read_csv(file) for file in file_list])
df.shape

/home/admin/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['./data/보험 상담_final_20250315_013253.csv', './data/강의_세미나_홍보글_final_20250315_015358.csv', './data/중고나라_게시물_final_20250315_015358.csv', './data/은행 상담_final_20250315_013253.csv', './data/이벤트_안내글_final_20250315_015358.csv', './data/모임_동호회_홍보글_final_20250315_015358.csv', './data/쇼핑 고객 서비스_final_20250315_013253.csv', './data/제품_리뷰글_final_20250315_015358.csv']


(807, 9)

In [2]:
df.head(2)

,origin_data,category,generate_score,generate_reason,anonymized_data,anonymized_prompt,validate_score,validate_reason,mapping
0,"상담사: ""안녕하세요, 김은주 고객님! 상담사 이현준입니다. 오늘 보험 상담 도와드...",보험 상담,5,"데이터는 모든 평가 기준을 완벽히 만족합니다. \n\n1. 개인정보 포함: 이름, ...","상담사: ""안녕하세요, [PERSON1] 고객님! 상담사 [PERSON2]입니다. ...",입력 데이터에 포함된 모든 개인정보를 위 placeholder를 사용하여 비식별화 ...,5,"모든 개인정보가 적절하게 비식별화되었습니다. 이름, 생년월일, 연락처, 주소, 이메...","{'김은주': '[PERSON1]', '이현준': '[PERSON2]', '1985..."
1,"상담사: ""안녕하세요, 김하나 고객님. 보험 상담을 도와드릴 김상진입니다. 생년월일...",보험 상담,5,데이터는 모든 평가 기준을 완벽히 만족하고 있습니다. \n1. 개인정보 포함: 이름...,"상담사: ""안녕하세요, [PERSON1] 고객님. 보험 상담을 도와드릴 [PERSO...",입력 데이터에 포함된 모든 개인정보를 위 placeholder를 사용하여 비식별화 ...,5,"모든 개인정보가 적절하게 비식별화되었으며, 동일한 개인정보는 일관된 placehol...","{'김하나': '[PERSON1]', '김상진': '[PERSON2]', '1985..."


In [3]:
# 학습한 모델을 경로를 지정합니다.
save_dir = "/home/admin/.cache/.work/model/model_5e-05_alpha-128_r-256"
peft_model_id = f"{save_dir}"

# PEFT 어댑터를 통해 사전 학습된 모델을 로드합니다.
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
).to("cuda")

# 토크나이저 로드합니다.
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
tokenizer.padding_side = 'right'  
tokenizer.pad_token = tokenizer.eos_token


Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.94s/it]


In [4]:
import datasets 

def get_chat_format(element):
    system_prompt = "너는 개인정보를 비식별화하는 Assistant야. 너는 주어진 데이터를 바탕으로 개인정보를 비식별화하는 작업을 해야해."

    return {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": element["origin_data"]},
            {"role": "assistant", "content": element["anonymized_data"]},
        ], 
        "label": element["mapping"]
    }

dataset = datasets.Dataset.from_pandas(df)
dataset = dataset.map(get_chat_format, remove_columns=dataset.features, batched=False)
dataset = dataset.shuffle(seed=42)
dataset = dataset.train_test_split(test_size=0.1, seed=42)

Map: 100%|██████████| 807/807 [00:00<00:00, 7543.46 examples/s]


In [42]:
dataset["test"][6]

{'messages': [{'content': '너는 개인정보를 비식별화하는 Assistant야. 너는 주어진 데이터를 바탕으로 개인정보를 비식별화하는 작업을 해야해.',
   'role': 'system'},
  {'content': '서울책사랑 동호회 안내\n\n안녕하세요, 책을 사랑하는 여러분! 서울책사랑 동호회에서 새로운 회원을 모집합니다. 저희 모임은 다양한 장르의 책을 함께 읽고 토론하며, 독서의 즐거움을 나누는 모임입니다. 독서뿐만 아니라 작가 초청 강연, 독서 여행 등 다양한 활동도 계획하고 있으니 많은 관심 부탁드립니다. 😊\n\n- 모임명: 서울책사랑 동호회\n- 활동 내용: 월 1회 정기 독서 모임, 작가 초청 강연, 독서 여행 등\n- 모임 일시: 매월 첫째 주 토요일 오후 3시\n- 장소: 서울시 종로구 종로3길 17, 3층 북카페\n- 가입 방법: 이메일로 간단한 자기소개와 함께 신청해주세요.\n- 연락처: 김민수 (회장) - 이메일: booklover@seoul.co.kr, 전화번호: 010-1234-5678\n- 소셜미디어 ID: 트위터 @seoulbooklovers\n\n새로운 인연을 만들고, 책과 함께하는 즐거운 시간을 보내고 싶으신 분들의 많은 참여 기다립니다. 독서를 통해 삶의 깊이를 더하고 싶으신 분이라면 누구나 환영합니다. 함께 책을 통해 세상을 더 넓게 바라보아요! 📚✨\n\n문의사항이 있으시면 언제든지 연락주세요. 감사합니다!',
   'role': 'user'},
  {'content': '서울책사랑 동호회 안내\n\n안녕하세요, 책을 사랑하는 여러분! `[LOCATION1]` 동호회에서 새로운 회원을 모집합니다. 저희 모임은 다양한 장르의 책을 함께 읽고 토론하며, 독서의 즐거움을 나누는 모임입니다. 독서뿐만 아니라 작가 초청 강연, 독서 여행 등 다양한 활동도 계획하고 있으니 많은 관심 부탁드립니다. 😊\n\n- 모임명: `[LOCATION1]` 동호회\n- 활동 내용: 월 1회 정기 독서 모임, 작가 초청 강연

In [132]:
dataset["test"][6]["messages"][1]["content"]

'서울책사랑 동호회 안내\n\n안녕하세요, 책을 사랑하는 여러분! 서울책사랑 동호회에서 새로운 회원을 모집합니다. 저희 모임은 다양한 장르의 책을 함께 읽고 토론하며, 독서의 즐거움을 나누는 모임입니다. 독서뿐만 아니라 작가 초청 강연, 독서 여행 등 다양한 활동도 계획하고 있으니 많은 관심 부탁드립니다. 😊\n\n- 모임명: 서울책사랑 동호회\n- 활동 내용: 월 1회 정기 독서 모임, 작가 초청 강연, 독서 여행 등\n- 모임 일시: 매월 첫째 주 토요일 오후 3시\n- 장소: 서울시 종로구 종로3길 17, 3층 북카페\n- 가입 방법: 이메일로 간단한 자기소개와 함께 신청해주세요.\n- 연락처: 김민수 (회장) - 이메일: booklover@seoul.co.kr, 전화번호: 010-1234-5678\n- 소셜미디어 ID: 트위터 @seoulbooklovers\n\n새로운 인연을 만들고, 책과 함께하는 즐거운 시간을 보내고 싶으신 분들의 많은 참여 기다립니다. 독서를 통해 삶의 깊이를 더하고 싶으신 분이라면 누구나 환영합니다. 함께 책을 통해 세상을 더 넓게 바라보아요! 📚✨\n\n문의사항이 있으시면 언제든지 연락주세요. 감사합니다!'

In [130]:
print(dataset["test"][6]["label"])


{'안녕하세요, 책을 사랑하는 여러분! 서울책사랑 동호회에서 새로운 회원을 모집합니다. 저희 모임은 다양한 장르의 책을 함께 읽고 토론하며, 독서의 즐거움을 나누는 모임입니다. 독서뿐만 아니라 작가 초청 강연, 독서 여행 등 다양한 활동도 계획하고 있으니 많은 관심 부탁드립니다. 😊': '[LOCATION1]', '- 모임명: 서울책사랑 동호회': '[LOCATION1]', '- 장소: 서울시 종로구 종로3길 17, 3층 북카페': '[ADDRESS1]', '- 연락처: 김민수 (회장) - 이메일: booklover@seoul.co.kr, 전화번호: 010-1234-5678': '[PERSON1]'}


In [ ]:
for conv_data in dataset["test"]:
    input_data = tokenizer.apply_chat_template(conv_data["messages"][:2], tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(input_data, return_tensors="pt").to("cuda")
    result = fine_tuned_model.generate(**inputs,
                            max_new_tokens=512, 
                            temperature=0.1,
                            pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(result[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
    break 


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [6]:
conv_data

{'messages': [{'content': '너는 개인정보를 비식별화하는 Assistant야. 너는 주어진 데이터를 바탕으로 개인정보를 비식별화하는 작업을 해야해.',
   'role': 'system'},
  {'content': '상담사: "안녕하세요, 홍길동 고객님. 저는 보험 상담사 김혜진입니다. 오늘 어떻게 도와드릴까요?"\n\n고객: "안녕하세요, 혜진 씨. 제가 최근에 이사해서 주소 변경을 하려고 합니다. 그리고 자동차 보험 갱신도 문의드리려고요."\n\n상담사: "네, 그럼 우선 주소 변경부터 도와드리겠습니다. 현재 주소가 \'서울시 강남구 테헤란로 123\'로 되어 있는데, 새 주소가 어떻게 되시나요?"\n\n고객: "네, 새 주소는 \'서울시 마포구 월드컵북로 456\'입니다."\n\n상담사: "확인했습니다. 시스템에 새 주소로 업데이트해드릴게요. 그리고 자동차 보험 갱신 관련 문의는 어떻게 도와드릴까요?"\n\n고객: "제가 1985년 5월 23일생인데, 이번에 갱신할 때 할인되는 부분이나 추가해야 할 특약이 있는지 궁금해요. 연락처는 010-1234-5678입니다."\n\n상담사: "네, 홍길동 고객님께서 쓰시는 차량이 현대 소나타 맞으시죠? 갱신하시면서 보상 한도를 조금 더 높이는 옵션을 고려하실 수 있습니다. 또한, 무사고 할인 혜택도 적용되실 수 있으니, 다음 갱신 때 조금 더 저렴한 비용으로 유지하실 수 있습니다."\n\n고객: "그렇군요. 혹시 보험료 납부 계좌를 변경할 수 있을까요? 지금은 국민은행 123-456-789012이거든요."\n\n상담사: "물론입니다. 새로운 계좌번호와 이름을 말씀해주시면 변경해 드리겠습니다."\n\n고객: "네, 새 계좌는 신한은행 987-654-321098입니다. 계좌명의는 홍길동으로 동일합니다."\n\n상담사: "확인했습니다. 변경 사항 모두 처리해드렸습니다. 추가로 문의사항 있으시면 언제든 연락주세요. 제 이메일은 hyejin.kim@insurancecompany.co.kr입니다."\n\n

In [7]:
raw_input = conv_data["messages"][1]["content"]
print(raw_input)

상담사: "안녕하세요, 홍길동 고객님. 저는 보험 상담사 김혜진입니다. 오늘 어떻게 도와드릴까요?"

고객: "안녕하세요, 혜진 씨. 제가 최근에 이사해서 주소 변경을 하려고 합니다. 그리고 자동차 보험 갱신도 문의드리려고요."

상담사: "네, 그럼 우선 주소 변경부터 도와드리겠습니다. 현재 주소가 '서울시 강남구 테헤란로 123'로 되어 있는데, 새 주소가 어떻게 되시나요?"

고객: "네, 새 주소는 '서울시 마포구 월드컵북로 456'입니다."

상담사: "확인했습니다. 시스템에 새 주소로 업데이트해드릴게요. 그리고 자동차 보험 갱신 관련 문의는 어떻게 도와드릴까요?"

고객: "제가 1985년 5월 23일생인데, 이번에 갱신할 때 할인되는 부분이나 추가해야 할 특약이 있는지 궁금해요. 연락처는 010-1234-5678입니다."

상담사: "네, 홍길동 고객님께서 쓰시는 차량이 현대 소나타 맞으시죠? 갱신하시면서 보상 한도를 조금 더 높이는 옵션을 고려하실 수 있습니다. 또한, 무사고 할인 혜택도 적용되실 수 있으니, 다음 갱신 때 조금 더 저렴한 비용으로 유지하실 수 있습니다."

고객: "그렇군요. 혹시 보험료 납부 계좌를 변경할 수 있을까요? 지금은 국민은행 123-456-789012이거든요."

상담사: "물론입니다. 새로운 계좌번호와 이름을 말씀해주시면 변경해 드리겠습니다."

고객: "네, 새 계좌는 신한은행 987-654-321098입니다. 계좌명의는 홍길동으로 동일합니다."

상담사: "확인했습니다. 변경 사항 모두 처리해드렸습니다. 추가로 문의사항 있으시면 언제든 연락주세요. 제 이메일은 hyejin.kim@insurancecompany.co.kr입니다."

고객: "감사합니다, 혜진 씨. 친절하게 도와주셔서 감사해요."

상담사: "별말씀을요! 좋은 하루 되세요."


In [8]:
print(output)

상담사: "안녕하세요, [PERSON1] 고객님. 저는 보험 상담사 [PERSON2]입니다. 오늘 어떻게 도와드릴까요?"

고객: "안녕하세요, [PERSON2] 씨. 제가 최근에 이사해서 주소 변경을 하려고 합니다. 그리고 자동차 보험 갱신도 문의드리려고요."

상담사: "네, 그럼 우선 주소 변경부터 도와드리겠습니다. 현재 주소가 '[ADDRESS1]'로 되어 있는데, 새 주소가 어떻게 되시나요?"

고객: "네, 새 주소는 '[ADDRESS2]'입니다."

상담사: "확인했습니다. 시스템에 새 주소로 업데이트해드릴게요. 그리고 자동차 보험 갱신 관련 문의는 어떻게 도와드릴까요?"

고객: "제가 [DATEOFBIRTH1]인데, 이번에 갱신할 때 할인되는 부분이나 추가해야 할 특약이 있는지 궁금해요. 연락처는 [CONTACT1]입니다."

상담사: "네, [PERSON1] 고객님께서 쓰시는 차량이 현대 소나타 맞으시죠? 갱신하시면서 보상 한도를 조금 더 높이는 옵션을 고려하실 수 있습니다. 또한, 무사고 할인 혜택도 적용되실 수 있으니, 다음 갱신 때 조금 더 저렴한 비용으로 유지하실 수 있습니다."

고객: "그렇군요. 혹시 보험료 납부 계좌를 변경할 수 있을까요? 지금은 [ACCOUNT1]이거든요."

상담사: "물론입니다. 새로운 계좌번호와 이름을 말씀해주시면 변경해 드리겠습니다."

고객: "네, 새 계좌는 [ACCOUNT2]입니다. 계좌명의는 [PERSON1]으로 동일합니다."

상담사: "확인했습니다. 변경 사항 모두 처리해드렸습니다. 추가로 문의사항 있으시면 언제든 연락주세요. 제 이메일은 [EMAIL1]입니다."

고객: "감사합니다, [PERSON2] 씨. 친절하게 도와주셔서 감사해요."

상담사: "별말씀을요! 좋은 하루 되세요."


In [9]:
def extract_placeholder_mapping(original_text, transformed_text, allowed_types):
    allowed_pattern = re.compile(r'\[(' + '|'.join(allowed_types) + r')\d*\]')
    generic_pattern = re.compile(r'(\[[^]]+\])')

    mapping = {}

    orig_lines = original_text.splitlines()
    trans_lines = transformed_text.splitlines()
    n_lines = min(len(orig_lines), len(trans_lines))

    for idx in range(n_lines):
        orig_line = orig_lines[idx]
        trans_line = trans_lines[idx]

        parts = re.split(generic_pattern, trans_line)
        orig_pos = 0

        for i, part in enumerate(parts):
            if allowed_pattern.match(part):
                # placeholder 발견
                # 다음 literal을 찾음
                next_literal = parts[i + 1] if i + 1 < len(parts) else ''
                
                # 다음 literal이 존재하면, 그 literal까지의 텍스트를 추출
                if next_literal:
                    next_idx = orig_line.find(next_literal, orig_pos)
                    if next_idx != -1:
                        replaced_text = orig_line[orig_pos:next_idx]
                        orig_pos = next_idx
                    else:
                        # 다음 literal을 못 찾으면 끝까지
                        replaced_text = orig_line[orig_pos:]
                        orig_pos = len(orig_line)
                else:
                    # 다음 literal이 없으면 남은 텍스트 전체
                    replaced_text = orig_line[orig_pos:]
                    orig_pos = len(orig_line)

                replaced_text = replaced_text.strip()
                if replaced_text:
                    mapping[replaced_text] = part

            else:
                # literal인 경우, 원본에서 위치 업데이트
                found_idx = orig_line.find(part, orig_pos)
                if found_idx != -1:
                    orig_pos = found_idx + len(part)

    return mapping

In [49]:
def extract_placeholder_mapping_fixed(original_text, transformed_text, allowed_types):
    allowed_pattern = re.compile(r'\[(' + '|'.join(allowed_types) + r')\d*\]')
    generic_pattern = re.compile(r'(\[[^]]+\])')

    mapping = {}

    orig_lines = original_text.splitlines()
    trans_lines = transformed_text.splitlines()
    n_lines = min(len(orig_lines), len(trans_lines))

    for idx in range(n_lines):
        orig_line = orig_lines[idx]
        trans_line = trans_lines[idx]

        parts = re.split(generic_pattern, trans_line)
        orig_pos = 0

        for i, part in enumerate(parts):
            if allowed_pattern.match(part):
                # placeholder 발견
                # 다음 literal을 찾음
                next_literal = parts[i + 1] if i + 1 < len(parts) else ''
                
                # 다음 literal이 존재하면, 그 literal까지의 텍스트를 추출
                if next_literal and len(next_literal.strip()) > 0:
                    # 공백을 무시하고 literal 찾기
                    next_literal_no_space = re.sub(r'\s+', '', next_literal)
                    orig_line_no_space = re.sub(r'\s+', '', orig_line[orig_pos:])
                    
                    next_idx_no_space = orig_line_no_space.find(next_literal_no_space)
                    
                    if next_idx_no_space != -1:
                        # 공백을 무시한 인덱스를 원래 인덱스로 변환
                        space_count = 0
                        for j in range(orig_pos, len(orig_line)):
                            if j - orig_pos - space_count == next_idx_no_space:
                                next_idx = j
                                break
                            if orig_line[j].isspace():
                                space_count += 1
                        
                        replaced_text = orig_line[orig_pos:next_idx].strip()
                        orig_pos = next_idx
                    else:
                        # 다음 literal을 못 찾으면, 최대 30자까지만 추출
                        replaced_text = orig_line[orig_pos:orig_pos+30].strip()
                        orig_pos += 30
                else:
                    # 다음 literal이 없으면, 최대 30자까지만 추출
                    replaced_text = orig_line[orig_pos:orig_pos+30].strip()
                    orig_pos += 30

                # 추출된 텍스트가 너무 길면 자르기
                if len(replaced_text) > 50:
                    replaced_text = replaced_text[:50].strip()

                if replaced_text:
                    mapping[replaced_text] = part
            else:
                # literal인 경우, 원본에서 위치 업데이트
                # 공백을 무시하고 literal 찾기
                part_no_space = re.sub(r'\s+', '', part)
                orig_line_no_space = re.sub(r'\s+', '', orig_line[orig_pos:])
                
                found_idx_no_space = orig_line_no_space.find(part_no_space)
                
                if found_idx_no_space != -1:
                    # 공백을 무시한 인덱스를 원래 인덱스로 변환
                    space_count = 0
                    for j in range(orig_pos, len(orig_line)):
                        if j - orig_pos - space_count == found_idx_no_space + len(part_no_space):
                            orig_pos = j
                            break
                        if orig_line[j].isspace():
                            space_count += 1

    return mapping

In [10]:
mapping_result = extract_placeholder_mapping(
    raw_input, 
    output, 
    allowed_types=(
        "PERSON", "CONTACT", "ADDRESS", "ACCOUNT", "DATEOFBIRTH", 
        "EMAIL", "LOCATION", "KAKO_ID", "TIWTTER_ID", "TELEGRAM_ID"))

print(mapping_result)

{'홍길동': '[PERSON1]', '김혜진': '[PERSON2]', '혜진': '[PERSON2]', '서울시 강남구 테헤란로 123': '[ADDRESS1]', '서울시 마포구 월드컵북로 456': '[ADDRESS2]', '1985년 5월 23일생': '[DATEOFBIRTH1]', '010-1234-5678': '[CONTACT1]', '국민은행 123-456-789012': '[ACCOUNT1]', '신한은행 987-654-321098': '[ACCOUNT2]', 'hyejin.kim@insurancecompany.co.kr': '[EMAIL1]'}


In [50]:
mapping_result = extract_placeholder_mapping_fixed(
    raw_input, 
    output, 
    allowed_types=(
        "PERSON", "CONTACT", "ADDRESS", "ACCOUNT", "DATEOFBIRTH", 
        "EMAIL", "LOCATION", "KAKO_ID", "TIWTTER_ID", "TELEGRAM_ID"))

print(mapping_result)

{'홍길동': '[PERSON1]', '김혜진': '[PERSON2]', '혜진': '[PERSON2]', '서울시 강남구 테헤란로 123': '[ADDRESS1]', '서울시 마포구 월드컵북로 456': '[ADDRESS2]', '1985년 5월 23일생': '[DATEOFBIRTH1]', '010-1234-5678': '[CONTACT1]', '국민은행 123-456-789012': '[ACCOUNT1]', '신한은행 987-654-321098': '[ACCOUNT2]', 'hyejin.kim@insurancecompany.co.kr': '[EMAIL1]'}


In [11]:
display(json_repair.loads(conv_data["label"]))

{'홍길동': '[PERSON1]',
 '김혜진': '[PERSON2]',
 '혜진': '[PERSON2]',
 '서울시 강남구 테헤란로 123': '[ADDRESS1]',
 '서울시 마포구 월드컵북로 456': '[ADDRESS2]',
 '1985년 5월 23일': '[DATEOFBIRTH1]',
 '010-1234-5678': '[CONTACT1]',
 '국민은행 123-456-789012': '[ACCOUNT1]',
 '신한은행 987-654-321098': '[ACCOUNT2]',
 'hyejin.kim@insurancecompany.co.kr': '[EMAIL1]'}

In [12]:
display(mapping_result)

{'홍길동': '[PERSON1]',
 '김혜진': '[PERSON2]',
 '혜진': '[PERSON2]',
 '서울시 강남구 테헤란로 123': '[ADDRESS1]',
 '서울시 마포구 월드컵북로 456': '[ADDRESS2]',
 '1985년 5월 23일생': '[DATEOFBIRTH1]',
 '010-1234-5678': '[CONTACT1]',
 '국민은행 123-456-789012': '[ACCOUNT1]',
 '신한은행 987-654-321098': '[ACCOUNT2]',
 'hyejin.kim@insurancecompany.co.kr': '[EMAIL1]'}

In [13]:
def compare_mappings(ground_truth, prediction, consider_partial_match=True):
    # 1. 키 집합 비교
    gt_keys = set(ground_truth.keys())
    pred_keys = set(prediction.keys())
    
    common_keys = gt_keys.intersection(pred_keys)
    only_in_gt = gt_keys - pred_keys
    only_in_pred = pred_keys - gt_keys
    
    # 2. 값 비교 (완전 일치)
    correct_values = sum(1 for k in common_keys if ground_truth[k] == prediction[k])
    
    # 3. 부분 일치 확인 및 처리
    partial_matches = []
    partial_correct = 0
    
    if consider_partial_match:
        for gt_key in list(only_in_gt):  # list로 변환하여 반복 중 삭제 가능하게 함
            for pred_key in list(only_in_pred):  # list로 변환하여 반복 중 삭제 가능하게 함
                # 부분 일치 조건 (한쪽이 다른 쪽의 부분 문자열이거나, 유사도가 높은 경우)
                if (gt_key in pred_key or pred_key in gt_key):
                    # 값도 같은지 확인
                    if ground_truth[gt_key] == prediction[pred_key]:
                        partial_matches.append((gt_key, pred_key))
                        partial_correct += 1
                        only_in_gt.remove(gt_key)
                        only_in_pred.remove(pred_key)
                        break
    
    # 4. 결과 출력
    print(f"완전 일치 키 수: {len(common_keys)}/{len(gt_keys)} ({len(common_keys)/len(gt_keys)*100:.2f}%)")
    print(f"값이 정확히 일치하는 키 수: {correct_values}/{len(common_keys)} ({correct_values/len(common_keys)*100 if common_keys else 0:.2f}%)")
    
    if consider_partial_match:
        print(f"부분 일치 키 쌍 수: {len(partial_matches)}/{len(gt_keys)} ({len(partial_matches)/len(gt_keys)*100:.2f}%)")
        print(f"부분 일치 키 쌍: {partial_matches}")
    
    print(f"GT에만 있는 키: {only_in_gt}")
    print(f"예측에만 있는 키: {only_in_pred}")
    
    # 5. 전체 정확도 계산 (완전 일치 + 부분 일치)
    total_correct = correct_values + partial_correct
    accuracy = total_correct / len(gt_keys) if gt_keys else 0
    print(f"전체 정확도(부분 일치 포함): {accuracy:.4f} ({total_correct}/{len(gt_keys)})")
    
    # 완전 일치만 고려한 정확도도 함께 제공
    strict_accuracy = correct_values / len(gt_keys) if gt_keys else 0
    print(f"엄격한 정확도(완전 일치만): {strict_accuracy:.4f} ({correct_values}/{len(gt_keys)})")
    
    return {
        "accuracy": accuracy,
        "strict_accuracy": strict_accuracy,
        "common_keys": common_keys,
        "correct_values": correct_values,
        "partial_matches": partial_matches,
        "partial_correct": partial_correct,
        "only_in_gt": only_in_gt,
        "only_in_pred": only_in_pred
    }
# 두 매핑 비교
gt_mapping = json_repair.loads(conv_data["label"])
pred_mapping = mapping_result

accuracy = compare_mappings(gt_mapping, pred_mapping)

완전 일치 키 수: 9/10 (90.00%)
값이 정확히 일치하는 키 수: 9/9 (100.00%)
부분 일치 키 쌍 수: 1/10 (10.00%)
부분 일치 키 쌍: [('1985년 5월 23일', '1985년 5월 23일생')]
GT에만 있는 키: set()
예측에만 있는 키: set()
전체 정확도(부분 일치 포함): 1.0000 (10/10)
엄격한 정확도(완전 일치만): 0.9000 (9/10)


In [109]:
def compare_mappings_simple(ground_truth, prediction, sample_idx=None, consider_partial_match=True):
    """
    정답 매핑과 예측 매핑을 비교하는 함수 (간결한 출력 버전)
    
    Args:
        ground_truth: 정답 매핑 딕셔너리
        prediction: 예측 매핑 딕셔너리
        sample_idx: 샘플 인덱스 (출력에 표시할 용도)
        consider_partial_match: 부분 일치를 고려할지 여부
    """
    # 1. 키 집합 비교
    gt_keys = set(ground_truth.keys())
    pred_keys = set(prediction.keys())
    
    common_keys = gt_keys.intersection(pred_keys)
    only_in_gt = gt_keys - pred_keys
    only_in_pred = pred_keys - gt_keys
    
    # 2. 값 비교 (완전 일치)
    correct_values = sum(1 for k in common_keys if ground_truth[k] == prediction[k])
    
    # 3. 부분 일치 확인 및 처리
    partial_matches = []
    partial_correct = 0
    
    if consider_partial_match:
        for gt_key in list(only_in_gt):
            for pred_key in list(only_in_pred):
                if (gt_key in pred_key or pred_key in gt_key):
                    if ground_truth[gt_key] == prediction[pred_key]:
                        partial_matches.append((gt_key, pred_key))
                        partial_correct += 1
                        only_in_gt.remove(gt_key)
                        only_in_pred.remove(pred_key)
                        break
    
    # 4. 결과 출력 - 간결한 형태로 변경
    sample_prefix = f"샘플 {sample_idx}: " if sample_idx is not None else ""
    print(f"{sample_prefix}완전 일치 키 수: {len(common_keys)}/{len(gt_keys)} ({len(common_keys)/len(gt_keys)*100:.2f}%)")
    print(f"값이 정확히 일치하는 키 수: {correct_values}/{len(common_keys)} ({correct_values/len(common_keys)*100 if common_keys else 0:.2f}%)")
    
    if consider_partial_match and partial_matches:
        print(f"부분 일치 키 쌍 수: {len(partial_matches)}/{len(gt_keys)} ({len(partial_matches)/len(gt_keys)*100:.2f}%)")
        print(f"부분 일치 키 쌍: {partial_matches}")
    
    # GT에만 있는 키와 예측에만 있는 키는 필요한 경우에만 출력
    if only_in_gt:
        print(f"GT에만 있는 키: {only_in_gt}")
    if only_in_pred:
        print(f"예측에만 있는 키: {only_in_pred}")
    
    # 5. 전체 정확도 계산 (완전 일치 + 부분 일치)
    total_correct = correct_values + partial_correct
    accuracy = total_correct / len(gt_keys) if gt_keys else 0
    print(f"전체 정확도(부분 일치 포함): {accuracy:.4f} ({total_correct}/{len(gt_keys)})")
    
    # 완전 일치만 고려한 정확도도 함께 제공
    strict_accuracy = correct_values / len(gt_keys) if gt_keys else 0
    print(f"엄격한 정확도(완전 일치만): {strict_accuracy:.4f} ({correct_values}/{len(gt_keys)})")
    
    return {
        "accuracy": accuracy,
        "strict_accuracy": strict_accuracy,
        "common_keys": common_keys,
        "correct_values": correct_values,
        "partial_matches": partial_matches,
        "partial_correct": partial_correct,
        "only_in_gt": only_in_gt,
        "only_in_pred": only_in_pred
    }

def aggregate_results_simple(results):
    """
    개별 샘플 평가 결과를 집계하는 함수 (간결한 출력 버전)
    """
    total_samples = len(results)
    total_accuracy = sum(r["evaluation"]["accuracy"] for r in results) / total_samples
    strict_accuracy = sum(r["evaluation"]["strict_accuracy"] for r in results) / total_samples
    
    # 부분 일치 케이스 집계 (샘플 인덱스 포함)
    all_partial_matches = []
    for r in results:
        for gt_key, pred_key in r["evaluation"]["partial_matches"]:
            all_partial_matches.append((r["sample_idx"], gt_key, pred_key))
    
    # 가장 자주 발생하는 부분 일치 패턴 (샘플 인덱스 포함)
    partial_match_patterns = {}
    for idx, gt_key, pred_key in all_partial_matches:
        pattern = f"샘플 {idx}: {gt_key} -> {pred_key}"
        partial_match_patterns[pattern] = partial_match_patterns.get(pattern, 0) + 1
    
    # 가장 자주 발생하는 오류 패턴 (샘플 인덱스 포함)
    error_patterns = {}
    for r in results:
        for gt_key in r["evaluation"]["only_in_gt"]:
            pattern = f"샘플 {r['sample_idx']}: 누락: {gt_key}"
            error_patterns[pattern] = error_patterns.get(pattern, 0) + 1
        for pred_key in r["evaluation"]["only_in_pred"]:
            pattern = f"샘플 {r['sample_idx']}: 추가: {pred_key}"
            error_patterns[pattern] = error_patterns.get(pattern, 0) + 1
    
    return {
        "total_samples": total_samples,
        "average_accuracy": total_accuracy,
        "average_strict_accuracy": strict_accuracy,
        "common_partial_matches": sorted(partial_match_patterns.items(), key=lambda x: x[1], reverse=True),
        "common_errors": sorted(error_patterns.items(), key=lambda x: x[1], reverse=True)
    }

def print_evaluation_summary_comprehensive(eval_results):
    """
    평가 결과 요약을 출력하는 함수 (종합적인 평가 지표 포함)
    """
    overall = eval_results["overall_results"]
    
    # 전체 데이터셋에 대한 정확도 및 추가 지표 계산
    total_gt_keys = 0
    total_pred_keys = 0
    total_correct_keys = 0
    total_partial_matches = 0
    
    # 개체 유형별 통계
    entity_type_stats = {}
    
    for r in eval_results["sample_results"]:
        gt_mapping = r["ground_truth_mapping"]
        pred_mapping = r["predicted_mapping"]
        
        total_gt_keys += len(gt_mapping)
        total_pred_keys += len(pred_mapping)
        total_correct_keys += r["evaluation"]["correct_values"]
        total_partial_matches += r["evaluation"]["partial_correct"]
        
        # 개체 유형별 통계 수집
        for key, value in gt_mapping.items():
            # 개체 유형 추출 (예: [PERSON1] -> PERSON)
            entity_type = re.match(r'\[([A-Z_]+)', value)
            if entity_type:
                entity_type = entity_type.group(1)
                if entity_type not in entity_type_stats:
                    entity_type_stats[entity_type] = {"gt": 0, "correct": 0, "partial": 0, "pred": 0}
                
                entity_type_stats[entity_type]["gt"] += 1
                
                # 완전 일치 확인
                if key in pred_mapping and pred_mapping[key] == value:
                    entity_type_stats[entity_type]["correct"] += 1
        
        # 예측된 개체 유형별 통계
        for key, value in pred_mapping.items():
            entity_type = re.match(r'\[([A-Z_]+)', value)
            if entity_type:
                entity_type = entity_type.group(1)
                if entity_type not in entity_type_stats:
                    entity_type_stats[entity_type] = {"gt": 0, "correct": 0, "partial": 0, "pred": 0}
                
                entity_type_stats[entity_type]["pred"] += 1
        
        # 부분 일치 개체 유형별 통계
        for gt_key, pred_key in r["evaluation"]["partial_matches"]:
            value = gt_mapping[gt_key]
            entity_type = re.match(r'\[([A-Z_]+)', value)
            if entity_type:
                entity_type = entity_type.group(1)
                if entity_type in entity_type_stats:
                    entity_type_stats[entity_type]["partial"] += 1
    
    # 정확도, 정밀도, 재현율, F1 점수 계산
    accuracy = (total_correct_keys + total_partial_matches) / total_gt_keys if total_gt_keys > 0 else 0
    strict_accuracy = total_correct_keys / total_gt_keys if total_gt_keys > 0 else 0
    
    precision = total_correct_keys / total_pred_keys if total_pred_keys > 0 else 0
    recall = total_correct_keys / total_gt_keys if total_gt_keys > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    # 부분 일치를 포함한 정밀도, 재현율, F1 점수
    precision_with_partial = (total_correct_keys + total_partial_matches) / total_pred_keys if total_pred_keys > 0 else 0
    recall_with_partial = (total_correct_keys + total_partial_matches) / total_gt_keys if total_gt_keys > 0 else 0
    f1_with_partial = 2 * precision_with_partial * recall_with_partial / (precision_with_partial + recall_with_partial) if (precision_with_partial + recall_with_partial) > 0 else 0
    
    print("=" * 70)
    print(f"총 평가 샘플 수: {overall['total_samples']}")
    
    # 전체 데이터셋 정확도 및 추가 지표 출력
    print("\n전체 데이터셋 평가 지표:")
    print(f"총 정답 키 수: {total_gt_keys}")
    print(f"총 예측 키 수: {total_pred_keys}")
    print(f"완전 일치 키 수: {total_correct_keys} ({total_correct_keys/total_gt_keys*100:.2f}%)")
    print(f"부분 일치 키 수: {total_partial_matches} ({total_partial_matches/total_gt_keys*100:.2f}%)")
    
    print("\n엄격한 평가 (완전 일치만):")
    print(f"정확도(Accuracy): {strict_accuracy:.4f}")
    print(f"정밀도(Precision): {precision:.4f}")
    print(f"재현율(Recall): {recall:.4f}")
    print(f"F1 점수: {f1:.4f}")
    
    print("\n유연한 평가 (부분 일치 포함):")
    print(f"정확도(Accuracy): {accuracy:.4f}")
    print(f"정밀도(Precision): {precision_with_partial:.4f}")
    print(f"재현율(Recall): {recall_with_partial:.4f}")
    print(f"F1 점수: {f1_with_partial:.4f}")
    
    # 개체 유형별 성능 출력
    print("\n개체 유형별 성능:")
    
    # 열 너비 설정
    col_widths = {
        "type": 15,
        "gt": 8,
        "pred": 8,
        "correct": 8,
        "partial": 8,
        "precision": 10,
        "recall": 10,
        "f1": 10
    }
    
    # 헤더 출력
    header = (
        f"{'개체 유형':<{col_widths['type']}}"
        f"{'정답 수':<{col_widths['gt']}}"
        f"{'예측 수':<{col_widths['pred']}}"
        f"{'정확 수':<{col_widths['correct']}}"
        f"{'부분 일치':<{col_widths['partial']}}"
        f"{'정밀도':<{col_widths['precision']}}"
        f"{'재현율':<{col_widths['recall']}}"
        f"{'F1 점수':<{col_widths['f1']}}"
    )
    print(header)
    print("-" * 80)
    
    for entity_type, stats in sorted(entity_type_stats.items()):
        gt_count = stats["gt"]
        pred_count = stats["pred"]
        correct_count = stats["correct"]
        partial_count = stats["partial"]
        
        # 개체 유형별 정밀도, 재현율, F1 점수 계산
        type_precision = correct_count / pred_count if pred_count > 0 else 0
        type_recall = correct_count / gt_count if gt_count > 0 else 0
        type_f1 = 2 * type_precision * type_recall / (type_precision + type_recall) if (type_precision + type_recall) > 0 else 0
        
        print(f"{entity_type:<15} {gt_count:<10} {pred_count:<10} {correct_count:<10} {partial_count:<10} {type_precision:.4f}    {type_recall:.4f}    {type_f1:.4f}")
    
    # 샘플별 평균 정확도 출력
    print("\n샘플별 평균 정확도:")
    print(f"평균 정확도 (부분 일치 포함): {overall['average_accuracy']:.4f}")
    print(f"평균 엄격한 정확도 (완전 일치만): {overall['average_strict_accuracy']:.4f}")
    
    # 부분 일치 패턴이 있는 경우에만 출력
    if overall["common_partial_matches"]:
        print("\n가장 흔한 부분 일치 패턴:")
        for pattern, count in overall["common_partial_matches"]:
            print(f"  - {pattern}: {count}회")
    
    # 오류 패턴이 있는 경우에만 출력
    if overall["common_errors"]:
        print("\n가장 흔한 오류 패턴:")
        for pattern, count in overall["common_errors"]:
            print(f"  - {pattern}: {count}회")
    print("=" * 70)

def evaluate_anonymization_model_simple(model, tokenizer, test_dataset, num_samples=None, temperature=0.1, max_new_tokens=512):
    """
    모델의 비식별화 성능을 평가하는 함수 (간결한 출력 버전)
    """
    # 경고 메시지 억제
    import warnings
    warnings.filterwarnings("ignore", message="Setting `pad_token_id` to")
    
    if num_samples is None:
        samples = test_dataset
    else:
        samples = test_dataset.select(range(min(num_samples, len(test_dataset))))
    
    results = []
    
    for idx, conv_data in enumerate(samples):
        print("------------------------------------")
        print(f"샘플 {idx+1}/{len(samples)} 평가 중...")
        
        # 입력 데이터 준비
        input_data = tokenizer.apply_chat_template(
            conv_data["messages"][:2],
            tokenize=False,
            add_generation_prompt=True
        )
        inputs = tokenizer(input_data, return_tensors="pt").to(model.device)
        
        # 모델 생성
        result = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id
        )
        
        # 생성된 텍스트 디코딩
        output = tokenizer.decode(result[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
        
        # 원본 입력 텍스트
        raw_input = conv_data["messages"][1]["content"]
        
        # 매핑 추출
        mapping_result = extract_placeholder_mapping(
            raw_input,
            output,
            allowed_types=(
                "PERSON", "CONTACT", "ADDRESS", "ACCOUNT", "DATEOFBIRTH",
                "EMAIL", "LOCATION", "KAKO_ID", "TWITTER_ID", "TELEGRAM_ID"
            )
        )
        
        # 정답 매핑
        gt_mapping = json_repair.loads(conv_data["label"])
        
        # 평가
        evaluation = compare_mappings_simple(gt_mapping, mapping_result, sample_idx=idx)
        
        # 결과 저장
        results.append({
            "sample_idx": idx,
            "raw_input": raw_input,
            "model_output": output,
            "ground_truth_mapping": gt_mapping,
            "predicted_mapping": mapping_result,
            "evaluation": evaluation
        })
    
    # 전체 결과 집계
    overall_results = aggregate_results_simple(results)
    
    return {
        "sample_results": results,
        "overall_results": overall_results
    }

# 사용 예시
eval_results = evaluate_anonymization_model_simple(fine_tuned_model, tokenizer, dataset["test"], num_samples=len(dataset["test"]))
print_evaluation_summary_comprehensive(eval_results)


------------------------------------
샘플 1/81 평가 중...
샘플 0: 완전 일치 키 수: 9/10 (90.00%)
값이 정확히 일치하는 키 수: 9/9 (100.00%)
부분 일치 키 쌍 수: 1/10 (10.00%)
부분 일치 키 쌍: [('1985년 5월 23일', '1985년 5월 23일생')]
전체 정확도(부분 일치 포함): 1.0000 (10/10)
엄격한 정확도(완전 일치만): 0.9000 (9/10)
------------------------------------
샘플 2/81 평가 중...
샘플 1: 완전 일치 키 수: 5/5 (100.00%)
값이 정확히 일치하는 키 수: 5/5 (100.00%)
전체 정확도(부분 일치 포함): 1.0000 (5/5)
엄격한 정확도(완전 일치만): 1.0000 (5/5)
------------------------------------
샘플 3/81 평가 중...
샘플 2: 완전 일치 키 수: 8/8 (100.00%)
값이 정확히 일치하는 키 수: 6/8 (75.00%)
전체 정확도(부분 일치 포함): 0.7500 (6/8)
엄격한 정확도(완전 일치만): 0.7500 (6/8)
------------------------------------
샘플 4/81 평가 중...
샘플 3: 완전 일치 키 수: 6/6 (100.00%)
값이 정확히 일치하는 키 수: 6/6 (100.00%)
전체 정확도(부분 일치 포함): 1.0000 (6/6)
엄격한 정확도(완전 일치만): 1.0000 (6/6)
------------------------------------
샘플 5/81 평가 중...
샘플 4: 완전 일치 키 수: 4/5 (80.00%)
값이 정확히 일치하는 키 수: 4/4 (100.00%)
GT에만 있는 키: {'5월 12일'}
예측에만 있는 키: {'KJH'}
전체 정확도(부분 일치 포함): 0.8000 (4/5)
엄격한 정확도(완전 일치만): 0.8000 (4/5)
----

In [123]:
print(eval_results["sample_results"][6]["raw_input"])
print("---------------------------------------------")
print(eval_results["sample_results"][6]["model_output"])
print("---------------------------------------------")
print(dataset["test"][6]["messages"][2]["content"])

서울책사랑 동호회 안내

안녕하세요, 책을 사랑하는 여러분! 서울책사랑 동호회에서 새로운 회원을 모집합니다. 저희 모임은 다양한 장르의 책을 함께 읽고 토론하며, 독서의 즐거움을 나누는 모임입니다. 독서뿐만 아니라 작가 초청 강연, 독서 여행 등 다양한 활동도 계획하고 있으니 많은 관심 부탁드립니다. 😊

- 모임명: 서울책사랑 동호회
- 활동 내용: 월 1회 정기 독서 모임, 작가 초청 강연, 독서 여행 등
- 모임 일시: 매월 첫째 주 토요일 오후 3시
- 장소: 서울시 종로구 종로3길 17, 3층 북카페
- 가입 방법: 이메일로 간단한 자기소개와 함께 신청해주세요.
- 연락처: 김민수 (회장) - 이메일: booklover@seoul.co.kr, 전화번호: 010-1234-5678
- 소셜미디어 ID: 트위터 @seoulbooklovers

새로운 인연을 만들고, 책과 함께하는 즐거운 시간을 보내고 싶으신 분들의 많은 참여 기다립니다. 독서를 통해 삶의 깊이를 더하고 싶으신 분이라면 누구나 환영합니다. 함께 책을 통해 세상을 더 넓게 바라보아요! 📚✨

문의사항이 있으시면 언제든지 연락주세요. 감사합니다!
---------------------------------------------
서울책사랑 동호회 안내

안녕하세요, 책을 사랑하는 여러분! 서울책사랑 동호회에서 새로운 회원을 모집합니다. 저희 모임은 다양한 장르의 책을 함께 읽고 토론하며, 독서의 즐거움을 나누는 모임입니다. 독서뿐만 아니라 작가 초청 강연, 독서 여행 등 다양한 활동도 계획하고 있으니 많은 관심 부탁드립니다. 😊

- 모임명: 서울책사랑 동호회
- 활동 내용: 월 1회 정기 독서 모임, 작가 초청 강연, 독서 여행 등
- 모임 일시: 매월 첫째 주 토요일 오후 3시
- 장소: [LOCATION1], 3층 북카페
- 가입 방법: 이메일로 간단한 자기소개와 함께 신청해주세요.
- 연락처: [PERSON1] (회장) - 이메일: [EMAIL1], 전화번호: [CONTACT1]
- 소셜미

In [ ]:
import re
import os
import json_repair
import pandas as pd
from openai import OpenAI
from pydantic import BaseModel
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv("./credit-env")
client = OpenAI(api_key=os.getenv("SELF_OPENAI_API_KEY"))


class OutputFormat(BaseModel):
    score: int  # 데이터 품질 평가 점수 (1~5)
    reason: str  # 해당 점수를 매긴 이유 (한국어로 설명)


def create_evaluation_prompt(original_text, ground_truth_anonymized, model_prediction):
    """
    LLM이 비식별화 성능을 평가하기 위한 프롬프트를 생성합니다.
    
    Args:
        original_text (str): 원본 텍스트
        ground_truth_anonymized (str): 정답 비식별화 텍스트
        model_prediction (str): 모델이 예측한 비식별화 텍스트
    
    Returns:
        str: 평가 프롬프트
    """
    
    prompt = f"""당신은 개인정보 비식별화 성능을 평가하는 전문가입니다. 원본 텍스트, 정답 비식별화 텍스트, 모델이 예측한 비식별화 텍스트를 분석하여 모델의 성능을 평가해주세요.

# 평가 데이터
## 원본 텍스트:
{original_text}

## 정답 비식별화 텍스트:
{ground_truth_anonymized}

## 모델 예측 비식별화 텍스트:
{model_prediction}

# 평가 지침
1. 개인정보란, 사람 이름, 연락처 (전화번호, 이메일, 카카오톡 ID 등), 주소(거주지), 계좌번호, 소셜미디어 ID (트위터, 텔레그램 등) 등을 지칭한다.  
2. 원본 텍스트에서 개인정보(이름, 주소, 연락처, 계좌번호, 이메일 등)를 모두 식별하세요.
3. 모델이 예측한 비식별화 텍스트에서 개인정보를 모두 식별하세요.
4. 정답 비식별화 텍스트와 모델이 예측한 비식별화 텍스트를 비교하여 모델이 모든 개인정보를 비식별화했는지를 평가하세요. 이때 원본 데이터가 동호회나 모임 등의 이름은 개인정보 했던 내용은 포함하지 마세요.
5. 사람 이름이 2명 등장 할때 정답은 홍길동을 PERSON1로 김철수를 PERSON2로 비식별화 하였는데, 모델을 홍길동을 PERSON2로 김철수를 PERSON1로 비식별화한 것은 문제가 되지 않습니다. 

5점 : 모델이 모든 개인정보를 비식별화했고, 정답 비식별화 텍스트와 모델이 예측한 비식별화 텍스트가 동일합니다.
4점 : 모델이 모든 개인정보를 비식별화한 경우
1점 : 모델이 개인정보를 비식별화하지 못한 경우

OUTPUT은 평가 점수와 그렇게 평가한 이유를 한국어로 출력하세요.
"""
    response = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "당신은 주어진 데이터를 평가하는 AI입니다. "},
        {"role": "user", "content": prompt}
    ],
    temperature=0.1,
    response_format=OutputFormat
    )
    
    return response

In [124]:
from tqdm.auto import tqdm 

openai_result_list = []
for idx in tqdm(range(len(dataset["test"]))):
    openai_result = create_evaluation_prompt(
        eval_results["sample_results"][idx]["raw_input"], 
        dataset["test"][idx]["messages"][2]["content"], 
        eval_results["sample_results"][idx]["model_output"]
        )
    openai_result_list.append(openai_result.choices[0].message.content)

print("모든 데이터를 openai에 평가하였습니다.")

100%|██████████| 81/81 [03:35<00:00,  2.67s/it]

모든 데이터를 openai에 평가하였습니다.


In [125]:
openai_result_list[0]

'{"score":4,"reason":"모델이 예측한 비식별화 텍스트는 모든 개인정보를 비식별화하는 데 성공했습니다. 원본 텍스트에서 식별된 개인정보(이름, 주소, 생년월일, 연락처, 계좌번호, 이메일)가 모두 적절히 비식별화되었습니다. 정답 비식별화 텍스트와 비교했을 때, 생년월일 부분에서 \'[DATEOFBIRTH1]생\' 대신 \'[DATEOFBIRTH1]\'로 표현된 차이가 있지만, 이는 비식별화의 정확성에 영향을 미치지 않습니다. 따라서 모델은 모든 개인정보를 비식별화했으므로 4점을 부여합니다."}'

In [126]:
import pandas as pd 

openai_df = pd.DataFrame(json_repair.loads(temp_data) for temp_data in openai_result_list)
openai_df.head(2)

,score,reason
0,4,모델이 예측한 비식별화 텍스트는 모든 개인정보를 비식별화하는 데 성공했습니다. 원본...
1,5,모델이 예측한 비식별화 텍스트는 정답 비식별화 텍스트와 완전히 동일합니다. 모든 개...


In [127]:
openai_df["score"].value_counts()

score
5    54
4    27
Name: count, dtype: int64